In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import matplotlib.pyplot as plt
import tqdm
import numpy as np

os.chdir("../src")
import simulator

In [58]:
S = simulator.Simulator(
    Ne = 100000,
    s = 1e-3,
    Un = 2e-3,
    Ud = 2e-3,
    n_generations = 10,
)

S.run()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 10.83it/s]
